# Implement a new method in pyVHR

Author :  Florian GIGOT

In this jupyter notebook, we are going to explain how to add its own method to the pyVHR framework.

This tutorial is based on my experience obtained during the integration of the "MAP 3DCNN" method in pyVHR. Thus, the implementation of "MAP 3DCNN" will be taken as an example to clarify the tutorial.

## Step 1 : method file creation

Firstly, you need to create a file with the name of your method in the "methods" package.

This file will define a class which will inherit from VHRMethod.

The "methodname" attribute defines the identifier of your method in the framework. 

The "apply" function will define the main function of the method.

In [ ]:
import numpy as np
from .base import VHRMethod

class MAP_3DCNN(VHRMethod):
    methodName = 'MAP_3DCNN'

    def __init__(self, **kwargs):
        super(MAP_3DCNN, self).__init__(**kwargs)

    def apply(self, X):
        return X

## Step 2 : link your method with the framework

To link your method with the rest of the framework, go to the "base.py" file of the "methods" package. Then, add your method to the "makeMethodObject" function.

In [ ]:
@staticmethod    
def makeMethodObject(video, methodName='ICA'):
    if methodName == 'CHROM':
        m = methods.CHROM(video)
        
    ...
        
    elif methodName == 'MAP_3DCNN': # My method id
        m = methods.MAP_3DCNN(video)
        
    ...
        
    else:
        raise ValueError("Unknown method!")
    return m

## Step 3 : specific configuration 

You can define a specific configuration for your method.

First of all, you just have to add your parameters in the configuration files ".cfg". The default configurations proposed by the authors of pyVHR are in the "analysis" package.

The syntax is as follows:

[ Method identifier]

parameter 1 = value 1

parameter 2 = value 2


etc.

In [ ]:
## Method specific configurations

# useRGBsig = disable the use of RGB signals
# useBVPsig = disable the use of BVP signals

## - MAP 3DCNN

# xstep = horizontal step for mapping
# ystep = vertical step for mapping
# modelFilename = path of the model

[MAP_3DCNN]
useRGBsig = 0 # parameter 1 
useBVPsig = 0 # parameter 2 
xstep     = 25 # parameter 3
ystep     = 25 # parameter 4
modelFilename = ./model_winsize5_mix/ # parameter 5

Then, you have to read the parameters into the framework.

Two ways to read a parameter exist:

* Define the reading in the "__readparams" function of the "base.py" file for generic parameters

* Define the reading in the constructor of the class of the method for specific parameters

In [ ]:
def __readparams(self, **kwargs):
    if 'useRGBsig' in kwargs:
        useRGBsig = int(kwargs['useRGBsig'])  # parameter 1
    else :
        useRGBsig = 1 #defaut value

    if 'useBVPsig' in kwargs:
        useBVPsig = int(kwargs['useBVPsig'])  # parameter 2
    else :
        useBVPsig = 1 #defaut value
            
    return startTime, endTime, winSize, timeStep, zeroMeanSTDnorm, BPfilter, minHz, maxHz,\
                detrending, detrMethod, detrLambda, useRGBsig, useBVPsig

In [ ]:
import numpy as np
from .base import VHRMethod

class MAP_3DCNN(VHRMethod):
    methodName = 'MAP_3DCNN'

    def __init__(self, **kwargs):
        self.x_step = int(kwargs['xstep']) # parameter 3
        self.y_step = int(kwargs['ystep'])  # parameter 4
        self.modelFilename = str(kwargs['modelFilename'])  # parameter 5
        super(MAP_3DCNN, self).__init__(**kwargs)

    def apply(self, X):
        bpm = np.asarray([80])
        return bpm 

## Step 4 : code your method

Now you have to write the code of your method in the "apply" function of your method file.

Some guidelines: 

* self : Object inheriting from VHRMethod

* X : sequence to process (by default the RGB signal)

* return : takes the following format "np.array([])" (by default the BVP signal) 


If your method respects the default template, you can stop the tutorial at the end of this step.


In [ ]:
import numpy as np
from .base import VHRMethod

class MAP_3DCNN(VHRMethod):
    methodName = 'MAP_3DCNN'

    def __init__(self, **kwargs):
        self.x_step = int(kwargs['xstep']) # parameter 3
        self.y_step = int(kwargs['ystep'])  # parameter 4
        self.modelFilename = str(kwargs['modelFilename'])  # parameter 5
        super(MAP_3DCNN, self).__init__(**kwargs)

    def apply(self, X):
        ##
        ## YOUR CODE
        ##
        return np.array([])

## Step 5 : adjustment

You can customise the input and output type of your function by adding conditions to the "runOffline" function in "base.py".  Plus, the signal processing functions are located in the "signals" package and can also be modified.

Please note that a bpm value must be added in bpmES for the framework to work properly.

Example specific to my method: 

In [ ]:
## methods/base.py/runOffline


# temporary variable initialization
rPPG = np.asarray([])
bpmEstimated = 0

...
 # -- use of RGB signals ??
if useRGBsig == 1 :
    ...
    
else:
    # cut video
    video = self.video.cutVideo(startFrame, endFrame)

    # apply method
    rPPG = self.apply(video)

    # memory management
    del video

# -- use of BVP signals ??
if useBVPsig == 1:           
    ...
else:
    bpmEstimated = rPPG

...
bpmES.append(bpmEstimated)
timesES.append(T)
...

In [ ]:
## signals/video.py

#  cut video
def cutVideo(self, startFrame, endFrame):
    if self.processedFaces.size == 0:
        return self.faces[startFrame:endFrame]
    else:
        return self.processedFaces[startFrame:endFrame]
